# Mapa de Rotas por Equipe usando OSRM + Folium

Este notebook:
- Lê um arquivo `.parquet` de resultados do V3 (por dia).
- Filtra por `dt_ref` e por `equipe`.
- Ordena as OS pela hora de chegada (`dth_chegada_estimada`).
- Usa **OSRM** para montar a rota real (base → OS1 → OS2 → ... → base), perna a perna.
- Plota tudo no **Folium**:
  - Rota rodoviária (quando o OSRM consegue calcular).
  - Linha reta como fallback quando alguma perna falha.
  - Marcadores numerados na ordem de atendimento, com popups detalhados.

🚚 PVLSN81 | 2023-01-27 06:00:00 → 15 OS (Tec=2 | Com=13) | 📦→ 78 (Tec=6 | Com=72)  
🚚 PVLSN84 | 2023-01-27 06:00:00 → 15 OS (Tec=3 | Com=12) | 📦→ 78 (Tec=6 | Com=72)  
🚚 PVOSN66 | 2023-01-27 06:00:00 → 15 OS (Tec=3 | Com=12) | 📦→ 78 (Tec=6 | Com=72)  
🚚 PVOSN67 | 2023-01-27 06:00:00 → 11 OS (Tec=3 | Com=8) | 📦→ 78 (Tec=6 | Com=72)  

In [52]:
import sys
import os
from pathlib import Path

import pandas as pd
import folium
from folium.plugins import PolyLineTextPath  # setas nas rotas
import requests
from itertools import cycle


In [53]:
# Caminho do arquivo parquet gerado pelo seu motor de roteirização
# Exemplo típico: "results_v3/atribuicoes_2023-01-01.parquet"
PARQUET_PATH = Path(r"E:\Rotas-Inteligentes\results_v4\atribuicoes_2023-01-01.parquet")

# Data (dt_ref) que você quer visualizar
# Use None para pegar todas as datas do arquivo
DIA_REF = "2023-01-01"    # ou None
BASE_LON = -63.885464691387746
BASE_LAT = -8.738508095069408

# ======================================================
# SELEÇÃO DE EQUIPES
# ======================================================
# Pode ser:
#   - None                     -> TODAS as equipes
#   - "PVLSN81"                -> UMA equipe específica (string)
#   - ["PVLSN81", "PVHRG02"]   -> CONJUNTO de equipes (lista/tupla/conjunto)
#
# Exemplos:
# EQUIPES_SELECIONADAS = None
# EQUIPES_SELECIONADAS = "PVLSN81"
# EQUIPES_SELECIONADAS = ["PVLSN81", "PVHRG02", "PVJPA01"]
EQUIPES_SELECIONADAS = None

# ======================================================
# Caminho do HTML de saída para salvar o mapa
# ======================================================
if DIA_REF and EQUIPES_SELECIONADAS is None:
    OUTPUT_HTML = Path(f"mapa_osrm_todas_equipes_{DIA_REF}.html")
elif DIA_REF and isinstance(EQUIPES_SELECIONADAS, str):
    OUTPUT_HTML = Path(f"mapa_osrm_eq_{EQUIPES_SELECIONADAS}_{DIA_REF}.html")
elif DIA_REF and not isinstance(EQUIPES_SELECIONADAS, str) and EQUIPES_SELECIONADAS is not None:
    OUTPUT_HTML = Path(f"mapa_osrm_conjunto_equipes_{DIA_REF}.html")
elif not DIA_REF and EQUIPES_SELECIONADAS is None:
    OUTPUT_HTML = Path("mapa_osrm_todas_equipes.html")
elif not DIA_REF and isinstance(EQUIPES_SELECIONADAS, str):
    OUTPUT_HTML = Path(f"mapa_osrm_eq_{EQUIPES_SELECIONADAS}.html")
else:
    OUTPUT_HTML = Path("mapa_osrm_conjunto_equipes.html")

# ======================================================
# URL do OSRM
# ======================================================
# 1) Tenta importar da sua aplicação (v2.config.OSRM_URL)
OSRM_URL = None
try:
    # Ajuste o path abaixo se necessário para apontar para a raiz do projeto
    sys.path.append(os.getcwd())
    from v2 import config as v2_config

    OSRM_URL = v2_config.OSRM_URL.rstrip("/")
    print(f"OSRM_URL carregada de v2.config: {OSRM_URL}")
except Exception:
    # 2) Se não conseguir importar, defina manualmente aqui:
    OSRM_URL = "http://localhost:5000"  # ajuste conforme a sua instância do OSRM
    print(f"Usando OSRM_URL manual: {OSRM_URL}")


Usando OSRM_URL manual: http://localhost:5000


In [54]:
# Carrega o parquet
df = pd.read_parquet(PARQUET_PATH, engine="pyarrow")
df = pd.DataFrame(df)  # garante que é um DataFrame do pandas
df["base_lat"] = BASE_LAT
df["base_lon"] = BASE_LON
# Verifica colunas necessárias
required_cols = {
    "tipo_serv",
    "numos",
    "datasol",
    "dataven",
    "datater_trab",
    "TD",
    "TE",
    "equipe",
    "dthaps_ini",
    "dthaps_fim_ajustado",
    "inicio_turno",
    "fim_turno",
    "dth_chegada_estimada",
    "dth_final_estimada",
    "fim_turno_estimado",
    "eta_source",
    "base_lon",
    "base_lat",
    "chegada_base",
    "latitude",
    "longitude",
    "dt_ref",
}

colsDate = [
    'datasol', 'dataven', 'datater_trab', 'dthaps_ini', 'dthaps_fim_ajustado',
    'inicio_turno', 'fim_turno', 'dthpausa_ini', 'dthpausa_fim',
    'dth_chegada_estimada', 'dth_final_estimada', 'fim_turno_estimado'
]

# Converter para datetime com coerce
df[colsDate] = df[colsDate].apply(
    lambda col: pd.to_datetime(col, errors='coerce')
)

# Arredondar para baixo no segundo
df[colsDate] = df[colsDate].apply(lambda col: col.dt.floor("s"))


missing_cols = required_cols - set(df.columns)
if missing_cols:
    raise ValueError(f"Colunas faltando no parquet: {missing_cols}")

# Normaliza dt_ref para apenas a data
df["dt_ref"] = pd.to_datetime(df["dt_ref"], errors="coerce").dt.date

# ===============================
# Filtro por dia de referência
# ===============================
if DIA_REF is not None:
    dia_ref_date = pd.to_datetime(DIA_REF).date()
    df = df[df["dt_ref"] == dia_ref_date].copy()

# ===============================
# Filtro por equipes selecionadas
# ===============================
if EQUIPES_SELECIONADAS is None:
    # TODAS as equipes
    df_filtrado = df.copy()
elif isinstance(EQUIPES_SELECIONADAS, str):
    # UMA equipe específica
    df_filtrado = df[df["equipe"] == EQUIPES_SELECIONADAS].copy()
else:
    # CONJUNTO de equipes (lista, tupla, conjunto, etc.)
    df_filtrado = df[df["equipe"].isin(EQUIPES_SELECIONADAS)].copy()

if df_filtrado.empty:
    msg_eq = "todas as equipes" if EQUIPES_SELECIONADAS is None else EQUIPES_SELECIONADAS
    raise ValueError(
        f"Nenhuma linha encontrada para seleção de equipes = {msg_eq}"
        + (f" no dia '{DIA_REF}'." if DIA_REF else ".")
    )

# Ordena por equipe e pela hora de chegada estimada
df_filtrado["dth_chegada_estimada"] = pd.to_datetime(
    df_filtrado["dth_chegada_estimada"], errors="coerce"
)
df_filtrado = (
    df_filtrado.sort_values(["equipe", "dth_chegada_estimada"])
    .reset_index(drop=True)
)

df_filtrado.head(10)


,tipo_serv,numos,datasol,dataven,datater_trab,TD,TE,equipe,dthaps_ini,dthaps_fim_ajustado,...,eta_source,base_lon,base_lat,chegada_base,latitude,longitude,dt_ref,EUSD,EUSD_FIO_B,job_id_vroom
0,técnico,20230000000543,2023-01-01 19:29:00,NaT,2023-01-01 23:47:56,56.42,34.03,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.755636,-63.901991,2023-01-01,146.85,146.85,14
1,técnico,20230000000512,2023-01-01 18:36:00,NaT,2023-01-01 20:37:28,15.93,14.68,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.762985,-63.901074,2023-01-01,12.63,12.63,6
2,comercial,93837792,2023-01-01 18:06:26,2023-01-03 08:00:00,2023-01-02 08:06:14,17.30,9.88,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.764141,-63.899824,2023-01-01,47.01,47.01,20
3,técnico,20230000000524,2023-01-01 18:48:39,NaT,2023-01-01 23:41:18,11.53,40.60,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.763655,-63.888055,2023-01-01,62.93,62.93,11
4,técnico,20230000000518,2023-01-01 18:42:51,NaT,2023-01-01 21:48:45,14.12,34.05,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.764398,-63.877153,2023-01-01,12.63,12.63,10
5,técnico,20230000000517,2023-01-01 18:40:43,NaT,2023-01-02 01:37:31,59.88,56.53,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.743574,-63.853778,2023-01-01,79.30,79.30,9
6,técnico,20230000001033,2023-01-01 19:44:00,NaT,2023-01-02 16:32:53,7.32,71.03,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.742915,-63.853234,2023-01-01,104.19,104.19,18
7,técnico,20230000000571,2023-01-01 19:06:00,NaT,2023-01-01 23:47:00,54.27,22.92,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.729593,-63.837551,2023-01-01,1252.87,1252.87,16
8,técnico,20230000000537,2023-01-01 19:09:36,NaT,2023-01-01 22:29:06,49.20,24.80,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.738911,-63.840107,2023-01-01,69.76,69.76,12
9,técnico,20230000000538,2023-01-01 19:11:58,NaT,2023-01-02 01:05:51,5.18,6.05,PVLSN06,NaT,NaT,...,VROOM,-63.885465,-8.738508,NaT,-8.741908,-63.856681,2023-01-01,215.71,215.71,13


In [55]:
def osrm_route_polyline_segmentado(coords_osrm, osrm_url: str):
    """
    Recebe uma lista de coordenadas (lon, lat) e o endereço do OSRM
    e retorna uma lista de pontos (lat, lon) para desenhar a rota no Folium.

    A rota é montada perna a perna:
    (p0 -> p1), (p1 -> p2), ..., (pn-1 -> pn)
    """
    if len(coords_osrm) < 2:
        return []

    osrm_url = osrm_url.rstrip("/")
    todos_pontos = []

    for (lon1, lat1), (lon2, lat2) in zip(coords_osrm[:-1], coords_osrm[1:]):
        url = f"{osrm_url}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
        params = {
            "overview": "full",
            "geometries": "geojson",
        }

        resp = requests.get(url, params=params, timeout=20)
        resp.raise_for_status()
        data = resp.json()

        routes = data.get("routes", [])
        if not routes:
            continue

        coords_seg = routes[0]["geometry"]["coordinates"]  # lista de [lon, lat]

        for i, (lon, lat) in enumerate(coords_seg):
            # evita repetir o primeiro ponto da perna, se já temos pontos acumulados
            if todos_pontos and i == 0:
                continue
            todos_pontos.append((lat, lon))  # Folium usa (lat, lon)

    return todos_pontos


In [ ]:
# Paleta de cores para as rotas das equipes
COLOR_PALETTE = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "darkred",
    "lightred",
    "beige",
    "darkblue",
    "darkgreen",
    "cadetblue",
    "darkpurple",
    "pink",
    "lightblue",
    "lightgreen",
    "gray",
    "black",
    "lightgray",
]


def construir_mapa_equipe_osrm(
    df_equipe: pd.DataFrame,
    equipe: str,
    osrm_url: str,
    cor_rota: str = "orange",
    mapa_existente=None,
) -> folium.Map:
    """
    Constrói (ou adiciona a) um mapa Folium para a equipe.

    - Base (base_lat/base_lon)
    - Serviços em ordem de chegada (dth_chegada_estimada)
    - Rota real via OSRM ligando base -> serviços -> base (perna a perna)
    - Ordem de atendimento numerada (1, 2, 3, ...)
    """

    df_equipe = df_equipe.copy()
    df_equipe["dth_chegada_estimada"] = pd.to_datetime(
        df_equipe["dth_chegada_estimada"], errors="coerce"
    )
    df_equipe = df_equipe.sort_values("dth_chegada_estimada").reset_index(drop=True)

    # Pega base (assumindo que é igual em todas as linhas da equipe)
    base_lon = float(df_equipe["base_lon"].iloc[0])
    base_lat = float(df_equipe["base_lat"].iloc[0])

    # Se não veio um mapa externo, cria um novo centralizado na base
    if mapa_existente is None:
        m = folium.Map(location=[base_lat, base_lon], zoom_start=12, tiles="OpenStreetMap")
    else:
        m = mapa_existente

    # Lista de pontos da rota: base -> serviços (na ordem) -> base
    coords_osrm = []
    coords_osrm.append((base_lon, base_lat))  # base inicial (lon, lat)

    # Serviços na ordem de chegada
    for _, row in df_equipe.iterrows():
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        coords_osrm.append((lon, lat))

    # Base final
    coords_osrm.append((base_lon, base_lat))

    # Marca a base da equipe (cor da rota)
    folium.CircleMarker(
        location=[base_lat, base_lon],
        radius=7,
        color=cor_rota,
        fill=True,
        fill_color=cor_rota,
        popup=f"Base equipe {equipe}",
        tooltip=f"Base {equipe}",
    ).add_to(m)

    # Chama o OSRM para pegar a geometria da rota
    try:
        linha_rota = osrm_route_polyline_segmentado(coords_osrm, osrm_url)
        if linha_rota:
            # Linha da rota com cor da equipe
            rota = folium.PolyLine(
                locations=linha_rota,
                color=cor_rota,
                weight=2,
                opacity=0.8,
                tooltip=f"Rota equipe {equipe}",
            ).add_to(m)

            # 🔹 Setas ao longo da rota (indicando o sentido)
            PolyLineTextPath(
                rota,
                "→.",          # caractere de seta
                repeat=True,
                offset=7,
                attributes={
                    "fill": cor_rota,
                    "font-weight": "thick",
                    "font-size": "6pt",
                },
            ).add_to(m)
        else:
            folium.Marker(
                location=[base_lat, base_lon],
                popup="OSRM não retornou geometria; rota não desenhada.",
                icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa"),
            ).add_to(m)
    except Exception as e:
        folium.Marker(
            location=[base_lat, base_lon],
            popup=f"Erro ao obter rota OSRM segmentada: {e}",
            icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa"),
        ).add_to(m)

    # Adiciona marcadores para cada OS, numerados na ordem de atendimento
    for ordem, (_, row) in enumerate(df_equipe.iterrows(), start=1):
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        numos = row["numos"]
        tipo = row["tipo_serv"]
        inicio_turno = row["inicio_turno"]
        apresentacao = row["dthaps_ini"]
        final_turno = row["fim_turno"]
        finalEstimado = row["fim_turno_estimado"]
        chegada = row["dth_chegada_estimada"]
        fim = row.get("dth_final_estimada", "")
        te = row.get("TE", "")
        td = row.get("TD", "")

        # 🔹 Popup destacando ORDEM e EQUIPE
        popup_html = (
            f"<b>Ordem de atendimento:</b> {ordem}<br>"
            f"<b>Equipe que atendeu:</b> {equipe}<br>"
            f"<b>OS:</b> {numos}<br>"
            f"<b>Tipo:</b> {tipo}<br>"
            f"<b>Chegada estimada:</b> {chegada}<br>"
            f"<b>Fim estimado:</b> {fim}<br>"
            f"<b>TE:</b> {te} min<br>"
            f"<b>TD:</b> {td} min<br>"
            f"<b>Início do turno:</b> {inicio_turno}<br>"
            f"<b>Apresentação:</b> {apresentacao}<br>"
            f"<b>Fim do turno:</b> {final_turno}<br>"
            f"<b>Fim do turno estimado:</b> {finalEstimado}<br>"
        )

        # Marcador da OS na cor da equipe
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color=cor_rota,
            fill=True,
            fill_color=cor_rota,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{equipe} | Ordem {ordem} - OS {numos} ({tipo})",
        ).add_to(m)

        # Rótulo com o número da ordem em cima do ponto
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 10pt; color: black;"><b>{ordem}</b></div>'
            ),
        ).add_to(m)

    return m


def construir_mapa_todas_equipes_osrm(
    df: pd.DataFrame,
    osrm_url: str,
) -> folium.Map:
    """
    Constrói um mapa com TODAS as equipes do dataframe,
    cada uma com uma cor de rota diferente.
    """
    df = df.copy()
    df["dth_chegada_estimada"] = pd.to_datetime(
        df["dth_chegada_estimada"], errors="coerce"
    )
    df = df.sort_values(["equipe", "dth_chegada_estimada"]).reset_index(drop=True)

    # Centro aproximado do mapa
    lat_centro = float(df["latitude"].mean())
    lon_centro = float(df["longitude"].mean())
    m = folium.Map(location=[lat_centro, lon_centro], zoom_start=8, tiles="OpenStreetMap")

    # Gera uma cor para cada equipe
    paleta = cycle(COLOR_PALETTE)
    cor_por_equipe = {}

    for equipe, df_eq in df.groupby("equipe"):
        cor = cor_por_equipe.setdefault(equipe, next(paleta))
        m = construir_mapa_equipe_osrm(
            df_equipe=df_eq,
            equipe=equipe,
            osrm_url=osrm_url,
            cor_rota=cor,
            mapa_existente=m,
        )

    return m


In [57]:
# Decide o que desenhar com base em EQUIPES_SELECIONADAS

if EQUIPES_SELECIONADAS is None:
    # TODAS as equipes do df_filtrado
    print("Construindo mapa com TODAS as equipes selecionadas (todas do filtro)...")
    mapa = construir_mapa_todas_equipes_osrm(df_filtrado, OSRM_URL)

elif isinstance(EQUIPES_SELECIONADAS, str):
    # UMA equipe específica
    print(f"Construindo mapa apenas para a equipe: {EQUIPES_SELECIONADAS}")
    df_eq = df_filtrado[df_filtrado["equipe"] == EQUIPES_SELECIONADAS].copy()
    mapa = construir_mapa_equipe_osrm(df_eq, EQUIPES_SELECIONADAS, OSRM_URL)

else:
    # CONJUNTO de equipes (já está filtrado em df_filtrado)
    lista_eq = sorted(set(df_filtrado["equipe"]))
    print(f"Construindo mapa para o CONJUNTO de equipes: {lista_eq}")
    mapa = construir_mapa_todas_equipes_osrm(df_filtrado, OSRM_URL)

# Salva o HTML
#mapa.save(OUTPUT_HTML)
print(f"Mapa salvo em: {OUTPUT_HTML}")

mapa


Construindo mapa com TODAS as equipes selecionadas (todas do filtro)...
Mapa salvo em: mapa_osrm_todas_equipes_2023-01-01.html


In [58]:
df.columns


Index(['tipo_serv', 'numos', 'datasol', 'dataven', 'datater_trab', 'TD', 'TE',
       'equipe', 'dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado', 'eta_source', 'base_lon',
       'base_lat', 'chegada_base', 'latitude', 'longitude', 'dt_ref', 'EUSD',
       'EUSD_FIO_B', 'job_id_vroom'],
      dtype='object')

In [59]:
colsDate =['datasol', 'dataven', 'datater_trab', 'dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado']
